In [1]:
# https://docs.binance.us/?python#get-candlestick-data
# binance.us
# Github.com - send Daniel Github username
# https://algotrading101.com/learn/binance-python-api-guide/
# https://data.binance.vision/
# https://github.com/sammchardy/python-binance

"""
Max requests per minute: 1200 (then IP ban)
Python-binance purposefully implements delays to prevent IP ban due to overloading
Some code is asyncronous
"""

from config import key, secret_key
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import numpy as np
import pandas as pd
import requests
from binance.enums import *
from binance.exceptions import BinanceAPIException, BinanceOrderException
from mysql.connector import connect
from db_config import host, port, username, password
connect.raise_on_warnings = False

client = Client(key, secret_key, tld='us') # Binance
connection = connect(host=host,
        user=username,
        password=password)
cursor = connection.cursor() # SQL

resp = requests.get('https://api.binance.us/api/v3/ping')
# resp = requests.get('https://api.binance.us/api/v3/klines?symbol=BTCUSD&interval=1m&limit=1000')
# print(pd.DataFrame(resp.json(), columns = ['open_time','open','high','low','close','volume','close_time','quote_asset_volume','number_of_trades','taker_buy_base_asset','taker_buy_quote_asset_volume', 'ignore']))

{}


In [ ]:
def to_unixmillis(from_date):
    from_date_obj = datetime.strptime(from_date, fmt)
    past = datetime(1970, 1, 1, tzinfo=from_date_obj.tzinfo)
    return int((from_date_obj - past).total_seconds() * 1000.0)


def to_datetime(ms):
    return datetime.fromtimestamp(int(float(ms) / 1000.0))

"""
BNB Coin allows for reduced commission fees. 25% cheaper
"""

def topup_bnb(min_balance: float, topup: float):
	''' Top up BNB balance if it drops below minimum specified balance '''
	bnb_balance = client.get_asset_balance(asset='BNB')
	bnb_balance = float(bnb_balance['free'])
	if bnb_balance < min_balance:
		qty = round(topup - bnb_balance, 5)
		print(qty)
		order = client.order_market_buy(symbol='BNBUSDT', quantity=qty)
		return order
	return False

In [3]:
# print(client.get_asset_balance(asset='BTC'))
# print(client.get_asset_balance(asset='USD'))
# get market depth
# depth = client.get_order_book(symbol='BTCUSD')

# place a test market buy order, to place an actual order use the create_order function
# order = client.create_test_order(
#     symbol='BNBBTC',
#     side=Client.SIDE_BUY,
#     type=Client.ORDER_TYPE_MARKET,
#     quantity=100)

# get latest price from Binance API
btc_price = client.get_symbol_ticker(symbol="BTCUSDT")
eth_price = client.get_symbol_ticker(symbol="ETHUSDT")
# print full output (dictionary)
print(btc_price)
print(eth_price)

In [5]:
# from time import sleep
# from datetime import datetime, timedelta
# import time
# from binance import ThreadedWebsocketManager

# btc_price = {'error':False}
# def btc_trade_history(msg):
#     ''' define how to process incoming WebSocket messages '''
#     if msg['e'] != 'error':
#         print(msg['c'])
#         btc_price['last'] = msg['c']
#         btc_price['bid'] = msg['b']
#         btc_price['last'] = msg['a']
#         btc_price['error'] = False
#     else:
#         btc_price['error'] = True

# bsm = ThreadedWebsocketManager()
# bsm.start()
# bsm.start_symbol_ticker_socket(callback=btc_trade_history, symbol='BTCUSDT')
# bsm.stop()

In [11]:
# valid intervals - 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

# get timestamp of earliest date data is available
start_date = client._get_earliest_valid_timestamp('BTCUSDT', '1d')
start_date = to_datetime(start_date)
fmt = "%Y-%m-%d %H:%M:%S"  # e.g. 2019-11-16 23:16:15
# end_date = time.strftime(fmt, time.localtime())
end_date = datetime.now()


print(start_date)
print(start_date+ timedelta(hours = 12))
print(end_date)


2019-09-22 20:00:00
2019-09-23 08:00:00
2022-04-09 14:18:44.518161


In [39]:
def binance_data_to_sql(start_date, end_date, currencytype, timespan, connection):
    cursor = connection.cursor()
    try:
        cursor.execute(f"""create table AlgoModule.{currencytype}_{timespan}
        (
        open_time datetime,
        close_time datetime,
        open_price double,
        close_price double,
        high_price double, 
        low_price double,
        volume double,
        quote_av double,
        trades int,
        tb_base_av double,
        tb_quote_av double
        )""")
        connection.commit()
    except:
        print("Error but moving on")

    step_date = start_date
    while step_date <= end_date:
        print(f'pulling {step_date} data for AlgoModule.{currencytype}_{timespan}')
        bars = client.get_historical_klines(currencytype, timespan, start_str = f'{step_date}', end_str=f'{step_date + timedelta(hours=12)}', limit=1000)
        bars = pd.DataFrame(bars, columns = ['date', 'open',
                    'high', 'low', 'close', 'volume', 'close_time', 'quote_av',
                    'trades', 'tb_base_av', 'tb_quote_av', 'ignore'])
        bars['date'] = bars['date'].apply(to_datetime)
        bars['close_time'] = bars['close_time'].apply(to_datetime)
        for index, row in bars.iterrows():
            cursor.execute(f"""INSERT INTO AlgoModule.{currencytype}_{timespan} (open_time, close_time, open_price, close_price, high_price, low_price,
            volume, quote_av, trades, tb_base_av, tb_quote_av) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
            , (row.date, row.close_time, row.open, row.close, row.high, row.low, row.volume, row.quote_av, row.trades, row.tb_base_av, row.tb_quote_av))
        step_date = step_date + timedelta(hours=12)
        connection.commit()
        sleep(1)

currency_pairings = ['ETHUSD'
# , 'BTCUSD'
]
time_intervals = [
    '1m' 
    # '3m',
    #  '5m', '15m', '30m', '1h', '2h', '4h', '6h',
    #  '8h', '12h'
    ]

for cur in currency_pairings:
    for t in time_intervals:
        binance_data_to_sql(start_date, end_date, cur, t, connection)


pulling 2019-09-22 20:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-23 08:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-23 20:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-24 08:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-24 20:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-25 08:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-25 20:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-26 08:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-26 20:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-27 08:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-27 20:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-28 08:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-28 20:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-29 08:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-29 20:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-30 08:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-09-30 20:00:00 data for AlgoModule.ETHUSD_5m
pulling 2019-1